# Widgets

<!-- SUMMARY: Demonstration of the use of widgets -->

<!-- CATEGORY: Tool_for_Python_environment -->

In [ ]:
import gstlearn as gl
import gstlearn.plot as gp
import gstlearn.document as gdoc
import gstlearn.widgets as gw

import matplotlib.pyplot as plt
import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
import math

The next two lines are compulsory if you want to use interactive graphics (matplotlib)

In [ ]:
%matplotlib inline
%matplotlib notebook

The next line avoids automatica scrolling of the jupyter-notebook to take place

In [ ]:
gdoc.setNoScroll()

## Defining the environment

Creating the Grid which will be used for all experiments

In [ ]:
nx = 120
db = gl.DbGrid.create(nx = [nx,nx],dx = [1,1])

Constitute the list of all available basic structures (starting from the Enum), restricted to the only ones that can be simulated using the Turning Bands method.

In [ ]:
models = gl.CovHelper.getAllCovariances(flagSimtub = True)
models

Defining the default values for main parameters

In [ ]:
ndat_ref = 10 # Number of samples
flagColor_ref = False # True for color representation; False for proportional representation
range_ref = 20  # Range of the structure
nbtuba_ref = 200 # Number of turning bands for the simulation
rank_ref = 1 # Rank of the basic structure (within 'models' list)
flagTitle_ref = True # Toggle flag for representing the Title

Defining the global parameters

In [ ]:
ndat = ndat_ref
flagColor = flagColor_ref
range = range_ref
nbtuba = nbtuba_ref
rank = rank_ref
flagTitle = flagTitle_ref

The following method resets the values of the global parameters to their default values. This method will be used in order to ensure that each chunk starts with the default values (rather than with the current valu of the global parameters).

In [ ]:
def reset():
    global ndat, flagColor, range, nbtuba, rank, flagTitle
    ndat = ndat_ref
    flagColor = flagColor_ref
    range = range_ref
    nbtuba = nbtuba_ref
    rank = rank_ref
    flagTitle = flagTitle_ref

Performing a non-conditional simulation (with Turning Bands method) and display the results. 

All these operations are set in a function (called **iteration**) as it will be called recursively for demonstration sake. This function is set with all variable arguments (where arguments are set by default to their initial values°.

In [ ]:
def iteration(ax):
    type = gl.ECov.fromKey(models[rank])
    model = gl.Model.createFromParam(type = type,range = range)
    db.deleteColumn("*Simu")
    err = gl.simtub(None,db,model,nbtuba=nbtuba)
    if flagTitle:
        title = f'{type.getDescr()} {":"} {"Range ="} {range} {"-"} {"Nbtuba ="} {nbtuba}'
    else:
        title = ""
        
    # Graphic representation
    ax.cla()
    ax.raster(db,"*Simu")
    ax.axes.axis("off")
    ax.decoration(title=title)
    
def graphicTest(ax):
    dbpoint = gl.Db.createFillRandom(ndat, 2, 1)
    
    ax.cla()
    if flagColor:
        ax.symbol(dbpoint, nameColor = "z", s=100)
    else:
        ax.symbol(dbpoint, nameSize="z")

## Testing interactive graphics

The first step is to test the graphic (matplotlib) in presence of widgets. Note that, in the next chunk, there is no widget involved.

The graphic simply displays a set of isolated points, simulated randomly. The display is performed either using  proportional (default) or color representation.

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(5,5))
reset()
graphicTest(ax)

In the next chunk, two widgets are involved. The information on the widgets (and their layout) will be provided in next paragraphs. There, we simply refer to the widgets and their main component: its eventHandler.

The first task is to define the **eventHandler** for each widget, that is the function which is called when the widget is modified. In this action, the task is simple:

- retrieve its new value
- update the global parameters
- trigger the figure

The second task is to instantiate the widgets (use one out of those provided by *gstlearn.widgets* library for example).

The last task is to instantiate the figure (using **plt.subplots** seems to be compulsory) and trigger the figure (using the global parameters).

Important remark: Using global variables (as demonstrated in this example) allows modifying each parameter of the plot, while keeping the other parameters unchanged.

In [ ]:
def EventNumberhandler(change):
    global ndat
    ndat = change.new
    graphicTest(ax)

def sliderColorEventhandler(change):
    global flagColor
    flagColor = change.new
    graphicTest(ax)

widgetNumber = gw.sliderInt(title='Number', value=ndat_ref, mini=1, maxi=100,
                        eventhandler=EventNumberhandler)
widgetColor = gw.boolean(title='Color?', value=flagColor_ref,
                         eventhandler=sliderColorEventhandler)
hbox = widgets.HBox([widgetNumber, widgetColor])

display(hbox)

fig,ax = plt.subplots(1,1,figsize=(5,5))
reset()
graphicTest(ax)

## Testing the Integer Slider

In [ ]:
def sliderNbtubaEventhandler(change):
    global nbtuba
    nbtuba = change.new
    iteration(ax)

widgetNbtuba = gw.sliderInt(title='Nb. Bands',
                            value=nbtuba_ref, mini=10, maxi=300,
                            eventhandler=sliderNbtubaEventhandler)

display(widgetNbtuba)

fig,ax = plt.subplots(1,1,figsize=(5,5))
reset()
iteration(ax)

## Testing the Float Slider

In [ ]:
def sliderRangeEventhandler(change):
    global range
    range = change.new
    iteration(ax)

widgetRange = gw.sliderFloat(title='Range',
                              value = range_ref, mini=5, maxi=100,
                              eventhandler=sliderRangeEventhandler)

display(widgetRange)

fig,ax = plt.subplots(1,1,figsize=(5,5))
reset()
iteration(ax)

## Testing the DropDown widget

In [ ]:
def sliderTypeEventhandler(change):
    global rank
    rank = models.index(change.new)
    iteration(ax)

widgetModelType = gw.dropDown(title='Structure', 
                              options = models, 
                              value = models[rank_ref],
                              eventhandler=sliderTypeEventhandler)

display(widgetModelType)

fig,ax = plt.subplots(1,1,figsize=(5,5))
reset()
iteration(ax)

## Testing the Boolean widget

Adding the possibility to hide the title

In [ ]:
def sliderTitleEventhandler(change):
    global flagTitle
    flagTitle = change.new
    iteration(ax)

widgetTitle = gw.boolean(title='Draw Title', value=True,
                         eventhandler=sliderTitleEventhandler)

display(widgetTitle)

fig,ax = plt.subplots(1,1,figsize=(5,5))
iteration(ax)

## Widget layout

In [ ]:
def sliderNbtubaEventhandler(change):
    global nbtuba
    nbtuba = change.new
    iteration(ax)
def sliderRangeEventhandler(change):
    global range
    range = change.new
    iteration(ax)

widgetNbtuba = gw.sliderInt(title='Nb. Bands',
                            value=nbtuba_ref, mini=10, maxi=300,
                            eventhandler=sliderNbtubaEventhandler)
widgetRange = gw.sliderFloat(title='Range',
                              value = range_ref, mini=5, maxi=100,
                              eventhandler=sliderRangeEventhandler)
hbox = widgets.HBox([widgetNbtuba, widgetRange])

display(hbox)

fig,ax = plt.subplots(1,1,figsize=(5,5))
iteration(ax)